# Inlined and Referenced Objects

## Replacing Identifiable Objects with their Identifiers

As an example, `Allele` contains a `location`, which must be `oneOf` 1) a subclass of `Location`, or 2) a `CURIE`.

Attributes that may be either objects or object identifiers are called _referable attributes_. When a referable attribute contains an object, the object is said to be _inlined_ ; when it is an identifier, it said to be _referenced_. 

`vr-python` provides code to convert referable attributes between their inlined and referenced forms. Replacing an inlined object with a referenced form is provided by `ga4gh_enref`, and replacing references with their inlined form is provided by `ga4gh_deref`.

Dereferencing an object requires having an _object store_ that converts GA4GH identifiers into GA4GH VR objects. In the simplest case, this may be a Python dictionary.  The `vr-python` implementation includes a Redis-backed object store that provides high-performance object retrieval.

![graphic](https://i.imgur.com/vBY3Pu8.png)

In [1]:
from ga4gh.core import ga4gh_deref, ga4gh_enref, ga4gh_identify
from ga4gh.vr import models, class_refatt_map

import json
def ppo(o):
    """pretty print object as json"""
    print(json.dumps(o.as_dict(), indent=2))

In [2]:
# A sample Allele
allele_dict = {
    'location': {
        'interval': {
            'end': 32936732,
            'start': 32936731,
            'type': 'SimpleInterval'},
        'sequence_id': 'ga4gh:SQ._0wi-qoDrvram155UmcSC-zA5ZK4fpLT',
        'type': 'SequenceLocation'},
    'state': {
        'sequence': 'A',
        'type': 'SequenceState'},
    'type': 'Allele'}

In [3]:
allele = models.Allele(**allele_dict)
allele._id = ga4gh_identify(allele)
allele.location._id = ga4gh_identify(allele.location)
ppo(allele)

{
  "_id": "ga4gh:VA.mpIbo0Vv4HT-Oh3g5SWcuzAR2mue3yL-",
  "location": {
    "_id": "ga4gh:VSL.v9K0mcjQVugxTDIcdi7GBJ_R6fZ1lsYq",
    "interval": {
      "end": 32936732,
      "start": 32936731,
      "type": "SimpleInterval"
    },
    "sequence_id": "ga4gh:SQ._0wi-qoDrvram155UmcSC-zA5ZK4fpLT",
    "type": "SequenceLocation"
  },
  "state": {
    "sequence": "A",
    "type": "SequenceState"
  },
  "type": "Allele"
}


## ga4gh_enref and ga4gh_deref using a Python dictionary

In [4]:
object_store = {}

In [5]:
allele_refd = ga4gh_enref(allele, cra_map=class_refatt_map, object_store=object_store)
ppo(allele_refd)

{
  "_id": "ga4gh:VA.mpIbo0Vv4HT-Oh3g5SWcuzAR2mue3yL-",
  "location": "ga4gh:VSL.v9K0mcjQVugxTDIcdi7GBJ_R6fZ1lsYq",
  "state": {
    "sequence": "A",
    "type": "SequenceState"
  },
  "type": "Allele"
}


In [6]:
object_store

{'ga4gh:VSL.v9K0mcjQVugxTDIcdi7GBJ_R6fZ1lsYq': <SequenceLocation _id=<Literal<str> ga4gh:VSL.v9K0mcjQVugxTDIcdi7GBJ_R6fZ1lsYq> interval=<SimpleInterval end=<Literal<int> 32936732> start=<Literal<int> 32936731> type=<Literal<str> SimpleInterval>> sequence_id=<Literal<str> ga4gh:SQ._0wi-qoDrvram155UmcSC-zA5ZK4fpLT> type=<Literal<str> SequenceLocation>>}

In [7]:
object_store["ga4gh:VSL.v9K0mcjQVugxTDIcdi7GBJ_R6fZ1lsYq"]

<SequenceLocation _id=<Literal<str> ga4gh:VSL.v9K0mcjQVugxTDIcdi7GBJ_R6fZ1lsYq> interval=<SimpleInterval end=<Literal<int> 32936732> start=<Literal<int> 32936731> type=<Literal<str> SimpleInterval>> sequence_id=<Literal<str> ga4gh:SQ._0wi-qoDrvram155UmcSC-zA5ZK4fpLT> type=<Literal<str> SequenceLocation>>

In [8]:
allele_inld = ga4gh_deref(allele_refd, cra_map=class_refatt_map, object_store=object_store)
ppo(allele_inld)

{
  "_id": "ga4gh:VA.mpIbo0Vv4HT-Oh3g5SWcuzAR2mue3yL-",
  "location": {
    "_id": "ga4gh:VSL.v9K0mcjQVugxTDIcdi7GBJ_R6fZ1lsYq",
    "interval": {
      "end": 32936732,
      "start": 32936731,
      "type": "SimpleInterval"
    },
    "sequence_id": "ga4gh:SQ._0wi-qoDrvram155UmcSC-zA5ZK4fpLT",
    "type": "SequenceLocation"
  },
  "state": {
    "sequence": "A",
    "type": "SequenceState"
  },
  "type": "Allele"
}


## ga4gh_enref and ga4gh_deref using a Redis database

The RedisObjectStore provides a dict-like interface (i.e., a `collections.abc.MutableMapping` interface) backed by Redis.

In [9]:
import redis
from ga4gh.vr.extras.redisobjectstore import RedisObjectStore

In [10]:
ros = RedisObjectStore(redis=redis.Redis(db=15), models=models)

In [11]:
allele_refd = ga4gh_enref(allele, cra_map=class_refatt_map, object_store=ros)
allele_refd.as_dict()

{'_id': 'ga4gh:VA.mpIbo0Vv4HT-Oh3g5SWcuzAR2mue3yL-',
 'location': 'ga4gh:VSL.v9K0mcjQVugxTDIcdi7GBJ_R6fZ1lsYq',
 'state': {'sequence': 'A', 'type': 'SequenceState'},
 'type': 'Allele'}

In [12]:
allele_inld = ga4gh_deref(allele_refd, cra_map=class_refatt_map, object_store=ros)
allele_inld.as_dict()

{'_id': 'ga4gh:VA.mpIbo0Vv4HT-Oh3g5SWcuzAR2mue3yL-',
 'location': {'_id': 'ga4gh:VSL.v9K0mcjQVugxTDIcdi7GBJ_R6fZ1lsYq',
  'interval': {'end': 32936732, 'start': 32936731, 'type': 'SimpleInterval'},
  'sequence_id': 'ga4gh:SQ._0wi-qoDrvram155UmcSC-zA5ZK4fpLT',
  'type': 'SequenceLocation'},
 'state': {'sequence': 'A', 'type': 'SequenceState'},
 'type': 'Allele'}

In [13]:
# Other objects can be stored also
ros[allele._id] = allele

In [14]:
list(ros.keys())

['ga4gh:VA.mpIbo0Vv4HT-Oh3g5SWcuzAR2mue3yL-',
 'ga4gh:VSL.v9K0mcjQVugxTDIcdi7GBJ_R6fZ1lsYq']